# Replication 0606

In [39]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from arch import arch_model
from sklearn.metrics import mean_squared_error

sp500 = pd.read_csv('composite_data.csv', parse_dates=['Date'], index_col='Date')
btc = pd.read_csv('bitcoin_bitstamp.csv', parse_dates=['Date'], index_col='Date')

sp500['Return'] = np.log(sp500['Close']) - np.log(sp500['Close'].shift(1))

btc['log_btc'] = np.log(btc['close'])

sp500['RV'] = sp500['Return'].rolling(window=20).apply(lambda x: np.sqrt(np.sum(x**2) * 252), raw=False)

data = pd.merge(sp500[['RV', "Return"]], btc[['log_btc']], left_index=True, right_index=True)
data.dropna(inplace=True)
data['const'] = 1

data.head()

,RV,Return,log_btc,const
Date,,,,
2017-12-21,0.308669,0.001984,9.655027,1
2017-12-22,0.307029,-0.000458,9.547512,1
2017-12-26,0.307428,-0.001059,9.665512,1
2017-12-27,0.265454,0.000791,9.639843,1
2017-12-28,0.266978,0.001832,9.579838,1


In [47]:
data['brk_1'] = (data.index >= pd.to_datetime('2019-02-01')).astype(int)
data['brk_2'] = (data.index >= pd.to_datetime('2020-02-27')).astype(int)
data['brk_3'] = (data.index >= pd.to_datetime('2020-10-08')).astype(int)
data['brk_4'] = (data.index >= pd.to_datetime('2021-05-21')).astype(int)

rho_model = sm.OLS(data['log_btc'].iloc[1:-h], data[['log_btc', 'const']].shift(1).iloc[1:-h]).fit()
rho = rho_model.params[0]
print(rho)

print(rho_model.summary())

0.9990109277530967
                            OLS Regression Results                            
Dep. Variable:                log_btc   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                 2.591e+05
Date:                Fri, 07 Jun 2024   Prob (F-statistic):               0.00
Time:                        16:34:09   Log-Likelihood:                 2142.9
No. Observations:                1236   AIC:                            -4282.
Df Residuals:                    1234   BIC:                            -4271.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
log_btc        0.9990      0.002 

In [48]:
data['predicted_log_btc'] = rho_model.predict(sm.add_constant(data['log_btc'].shift(1)))

data.head()

,RV,Return,log_btc,const,brk_1,brk_2,brk_3,brk_4,predicted_log_btc
Date,,,,,,,,,
2017-12-21,0.308669,0.001984,9.655027,1,0,0,0,0,NaN
2017-12-22,0.307029,-0.000458,9.547512,1,0,0,0,0,1.092816
2017-12-26,0.307428,-0.001059,9.665512,1,0,0,0,0,1.091772
2017-12-27,0.265454,0.000791,9.639843,1,0,0,0,0,1.092918
2017-12-28,0.266978,0.001832,9.579838,1,0,0,0,0,1.092669


In [49]:
data['log_btc_lag1'] = data['log_btc'].shift(1)
data['btc_diff'] = data['predicted_log_btc'] - rho * data['log_btc_lag1'] # data['log_btc'] - rho * data['log_btc_lag1']
data = data.dropna()

# split_date = data.index[int(len(data) * 0.75)]
h = 120
train_data = data[:-h]
test_data = data[-h:]

X = train_data[['const', 'log_btc_lag1', 'btc_diff', 'brk_1', 'brk_2', 'brk_3', 'brk_4']] # , 'brk_4'
y = train_data['RV']

model = sm.OLS(y, X).fit()
print(model.summary())

# GARCH(1,1) 모델 적용
# am = arch_model(train_data['RV'], vol='Garch', p=1, q=1)
# res = am.fit()
# print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                     RV   R-squared:                       0.417
Model:                            OLS   Adj. R-squared:                  0.414
Method:                 Least Squares   F-statistic:                     175.8
Date:                Fri, 07 Jun 2024   Prob (F-statistic):          3.00e-141
Time:                        16:34:39   Log-Likelihood:                -1032.3
No. Observations:                1236   AIC:                             2077.
Df Residuals:                    1230   BIC:                             2107.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            3.1415      0.260     12.088   

In [51]:
y_pred_train = model.predict(X)

X_2 = test_data[['const', 'log_btc_lag1', 'btc_diff', 'brk_1', 'brk_2', 'brk_3', 'brk_4']] # , 'brk_4'
# X_2['const'] = 1
y_test = model.predict(X_2)
y_test.head()

Date
2021-07-12    0.498554
2021-07-13    0.509016
2021-07-14    0.514575
2021-07-15    0.513160
2021-07-16    0.527792
dtype: float64

In [55]:
forecast_horizons = [120]

panel_b_results = {}
panel_c_results = {}

for h in forecast_horizons:
    y_pred_list = []
    btc_pred = test_data['log_btc'].iloc[0]

    for i in range(h):
        if i == 0:
            btc_lag = train_data['log_btc'].iloc[-1]
        else:
            btc_lag = btc_pred

        btc_diff = btc_pred - rho * btc_lag
        X_new = sm.add_constant(pd.DataFrame({
            'log_btc_lag1': [btc_lag],
            'btc_diff': [btc_diff],
            'brk_1': [1 if test_data.index[i] >= pd.to_datetime('2019-02-01') else 0],
            'brk_2': [1 if test_data.index[i] >= pd.to_datetime('2020-02-27') else 0],
            'brk_3': [1 if test_data.index[i] >= pd.to_datetime('2020-10-08') else 0],
            'brk_4': [1 if test_data.index[i] >= pd.to_datetime('2021-05-21') else 0],
        }))
        X_new['const'] = 1
        y_pred = model.predict(X_new)
        print(y_pred)
        y_pred_list.append(y_pred.iloc[0])

        btc_pred = model.predict(X_new).iloc[0]

    y_pred = np.array(y_pred_list)
    y_test_h = test_data['RV'].iloc[-h:].values

    y_benchmark = np.full_like(y_test_h, train_data['RV'].mean())

    rmse_model = np.sqrt(mean_squared_error(y_test_h, y_pred))
    rmse_benchmark = np.sqrt(mean_squared_error(y_test_h, y_benchmark))

    relative_rmse = rmse_model / rmse_benchmark
    panel_b_results[f'Relative RMSE (Out-of-Sample h={h})'] = relative_rmse

    f_t = (y_test_h - y_pred)**2 - (y_test_h - y_benchmark)**2
    cw_stat = np.mean(f_t) / np.std(f_t, ddof=1) * np.sqrt(len(f_t))
    print("mean f_t is ", np.mean(f_t))
    panel_c_results[f'CW (Out-of-sample h={h})'] = cw_stat

print("Panel B Results:", panel_b_results)
print("Panel C Results:", panel_c_results)

0    35.167473
dtype: float64
0    112.943356
dtype: float64
0    357.375086
dtype: float64
0    1125.567852
dtype: float64
0    3539.821219
dtype: float64
0    11127.264856
dtype: float64
0    34972.856923
dtype: float64
0    109914.074059
dtype: float64
0    345437.101119
dtype: float64
0    1.085632e+06
dtype: float64
0    3.411894e+06
dtype: float64
0    1.072280e+07
dtype: float64
0    3.369930e+07
dtype: float64
0    1.059092e+08
dtype: float64
0    3.328482e+08
dtype: float64
0    1.046065e+09
dtype: float64
0    3.287543e+09
dtype: float64
0    1.033199e+10
dtype: float64
0    3.247108e+10
dtype: float64
0    1.020491e+11
dtype: float64
0    3.207170e+11
dtype: float64
0    1.007940e+12
dtype: float64
0    3.167723e+12
dtype: float64
0    9.955427e+12
dtype: float64
0    3.128762e+13
dtype: float64
0    9.832980e+13
dtype: float64
0    3.090280e+14
dtype: float64
0    9.712039e+14
dtype: float64
0    3.052271e+15
dtype: float64
0    9.592585e+15
dtype: float64
0    3.014729e+16

In [28]:
import numpy as np

risk_free_rate = 0.02 / 252

gamma, theta = 3, 6

def calculate_portfolio_returns(predicted_volatility, actual_returns, risk_free_rate):
    def calculate_weight(predicted_return, risk_free_rate, predicted_volatility, theta, gamma):
        w_t = (1 / gamma) * (theta * predicted_return + (theta - 1) * risk_free_rate) / (theta ** 2 * predicted_volatility ** 2)
        return w_t
    w_t = calculate_weight(y_pred, risk_free_rate, predicted_volatility, theta, gamma)
    portfolio_returns = w_t * actual_returns + (1 - w_t) * risk_free_rate
    return portfolio_returns

portfolio_returns_wn = calculate_portfolio_returns(y_pred, test_data["Return"], risk_free_rate)
portfolio_returns_ha = calculate_portfolio_returns(np.full_like(y_test, np.mean(train_data['RV'])), test_data["Return"], risk_free_rate)

annual_factor = 252

geomean = 1
for r in portfolio_returns_wn :
    geomean = geomean *(1+r)
geomean = geomean ** (1/len(portfolio_returns_wn))
geomean1 = geomean - 1

geomean = 1
for r in portfolio_returns_ha :
    geomean = geomean *(1+r)
geomean = geomean ** (1/len(portfolio_returns_ha))
geomean2 = geomean - 1

annual_return_wn = geomean1 * annual_factor
# annual_return_wn = np.mean(portfolio_returns_wn) * annual_factor
annual_volatility_wn = np.std(portfolio_returns_wn) * np.sqrt(annual_factor)
annual_return_ha = geomean2 * annual_factor
# annual_return_ha = np.mean(portfolio_returns_ha) * annual_factor
annual_volatility_ha = np.std(portfolio_returns_ha) * np.sqrt(annual_factor)

def calculate_cer(returns, gamma):
    mean_return = np.mean(returns) 
    var_return = np.var(returns) 
    return (mean_return - 0.5 * gamma * var_return) * annual_factor

cer_wn_gamma_3_theta_6 = calculate_cer(portfolio_returns_wn, 3)
cer_ha_gamma_3_theta_6 = calculate_cer(portfolio_returns_ha, 3)

def calculate_sharpe_ratio(returns, std, risk_free_rate=risk_free_rate):
    excess_returns = returns - risk_free_rate
    return np.mean(excess_returns) / std# * np.sqrt(annual_factor)

sharpe_ratio_wn = calculate_sharpe_ratio(portfolio_returns_wn, annual_volatility_wn)
sharpe_ratio_ha = calculate_sharpe_ratio(portfolio_returns_ha, annual_volatility_ha)

print("WN Model:")
print(f"Annual Return: {annual_return_wn}")
print(f"Annual Volatility: {annual_volatility_wn}")
print(f"CER (Gamma=3, Theta=6): {cer_wn_gamma_3_theta_6}")
print(f"Sharpe Ratio: {sharpe_ratio_wn}")

print("\nHA Model:")
print(f"Annual Return: {annual_return_ha}")
print(f"Annual Volatility: {annual_volatility_ha}")
print(f"CER (Gamma=3, Theta=6): {cer_ha_gamma_3_theta_6}")
print(f"Sharpe Ratio: {sharpe_ratio_ha}")


WN Model:
Annual Return: 0.029895875050350185
Annual Volatility: 0.01094020820240655
CER (Gamma=3, Theta=6): 0.029776182520768583
Sharpe Ratio: 0.003611156556492211

HA Model:
Annual Return: 0.2697922163377582
Annual Volatility: 0.09780784071979298
CER (Gamma=3, Theta=6): 0.26013005376178167
Sharpe Ratio: 0.010324731963594225


In [25]:
# forecast_horizons = [30, 60, 120]

# panel_b_results = {}
# panel_c_results = {}

# for h in forecast_horizons:
#     X_test = test_data[['log_btc_lag1', 'btc_diff', 'brk_1', 'brk_2', 'brk_3', 'brk_4']]
#     X_test['const'] = 1
#     y_test = test_data['RV']

#     y_pred = model.predict(X_test)[:h]
#     y_test_h = y_test[:h]

#     y_benchmark = np.full_like(y_test_h, train_data['RV'].mean())

#     rmse_model = np.sqrt(mean_squared_error(y_test_h, y_pred))
#     rmse_benchmark = np.sqrt(mean_squared_error(y_test_h, y_benchmark))

#     relative_rmse = rmse_model / rmse_benchmark
#     print(y_pred.head())
#     print(y_test_h.head())
#     panel_b_results[f'Relative RMSE (Out-of-Sample h={h})'] = relative_rmse

#     f_t = (y_test_h - y_pred)**2 - (y_test_h - y_benchmark)**2
#     cw_stat = np.mean(f_t) / np.std(f_t, ddof=1) * np.sqrt(len(f_t))
#     panel_c_results[f'CW (Out-of-sample h={h})'] = cw_stat

# print("Panel B Results:", panel_b_results)
# print("Panel C Results:", panel_c_results)

## Bai and Perron (2003)

In [20]:
from ruptures import Binseg

algo = Binseg(model="l2").fit(data['RV'].to_frame())
breakpoints = algo.predict(n_bkps=4)

breakpoints = list(map(lambda i: i-1, breakpoints))

print(breakpoints)

breakpoint_dates = data.index[breakpoints]
print(breakpoint_dates)

[764, 789, 834, 854, 1354]
DatetimeIndex(['2020-02-24', '2020-03-11', '2020-04-14', '2020-04-28',
               '2021-12-29'],
              dtype='datetime64[ns]', name='Date', freq=None)
